In [20]:
import asana
from asana.rest import ApiException
from pprint import pprint
import os
import requests
from helpers import get_gid_from_json
import pandas as pd
from datetime import datetime

import psycopg2
from sqlalchemy import create_engine, text, MetaData, Table
from dotenv import load_dotenv, dotenv_values

In [21]:

gid_file_path = 'workspace.json'

load_dotenv()
access_token = os.getenv('ASANA_TOKEN')
workspace_gid = get_gid_from_json(gid_file_path)




configuration = asana.Configuration()
configuration.access_token = access_token
api_client = asana.ApiClient(configuration)


# create an instance of the API class
workspaces_api_instance = asana.WorkspacesApi(api_client)
opts_workspaces = {
    'opt_fields': "https://app.asana.com/api/1.0/workspaces" # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
}


def get_workspace_gid(opts_workspaces):
    try:
        # Get multiple workspaces
        api_response = workspaces_api_instance.get_workspaces(opts_workspaces)
        for data in api_response:
            workspace_gid = int(data['gid'])
            print(f'Workspace GID: {workspace_gid}')
            return workspace_gid
    except ApiException as e:
        print("Exception when calling WorkspacesApi->get_workspaces: %s\n" % e)
        
        
workspace_gid = get_workspace_gid(opts_workspaces)



# create an instance of the API class
projects_api_instance = asana.ProjectsApi(api_client)
opts_project = {
    'workspace': workspace_gid # str | The workspace or organization to filter projects on.
}


def get_project_gid(opts_project):
    try:
        # Get multiple projects
        api_response = projects_api_instance.get_projects(opts_project)
        for data in api_response:
            project_name = data['name']
            project_gid = data['gid']
            
            project_details = {
                'project_name': project_name,
                'project_gid': project_gid
            }
            
            print(f'Project Name: {project_name}')
            print(f'Project GID: {project_gid}')

            return project_details
            
    except ApiException as e:
        print("Exception when calling ProjectsApi->get_projects: %s\n" % e)
        
        
project_details = get_project_gid(opts_project)






# create an instance of the API class
tasks_api_instance = asana.TasksApi(api_client)
project_gid = project_details['project_gid'] # str | Globally unique identifier for the project.
    
    
opts_tasks = {
    'opt_fields': "notes,created_at,memberships.section.name,assignee.name,tags.name,name,created_at,f'https://app.asana.com/api/1.0/projects/{project_gid}/tasks'", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
#     'opt_fields': "assignee.name,section,tags.name,name,f'https://app.asana.com/api/1.0/projects/{project_gid}/tasks'", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.

}


def get_all_task(project_gid, opts_tasks):
    try:
        # Get tasks from a project
        api_response = tasks_api_instance.get_tasks_for_project(project_gid, opts_tasks)
        data_list = []
        for data in api_response:
            data_list.append(data)
        
        df = pd.DataFrame(data_list)
        
        return df
    
    except ApiException as e:
        print("Exception when calling TasksApi->get_tasks_for_project: %s\n" % e)
        
df = get_all_task(project_gid, opts_tasks)



Workspace GID: 1207998439135084
Project Name: AMM & MPLAN Enrollment
Project GID: 1207998440065110


In [22]:

def rename_column_names(df):
    column_mapping = {
        'created_at': 'INIT DATE',
        'memberships': 'STATUS',
        'assignee': 'PIC',
        'name': 'DESCRIPTION',
        'notes': 'PART / MPLAN NO.'
    }
    df = df.rename(columns=column_mapping)
    return df



df = rename_column_names(df)


# Function to extract the name from the email or handle None/empty cases
def extract_name(assignee_dict):
    if assignee_dict is None:
        return "<unassigned>"
    else:
        email = assignee_dict['name']
        first_name, last_name = email.split('@')[0].split('.')
        return f"{first_name.capitalize()} {last_name.capitalize()}"

# Apply the function to the 'assignee' column
df['PIC'] = df['PIC'].apply(lambda x: extract_name(x) if x else "<unassigned>")

def extract_date(timestamp):
    if pd.isna(timestamp) or timestamp == '':
        return ''
    return timestamp.split('T')[0]

# Apply the function to the 'created_at' column
# df['created_at'] = df['created_at'].apply(lambda x: extract_date(x))
df['INIT DATE'] = df['INIT DATE'].apply(lambda x: extract_date(x))
df['INIT DATE'] = pd.to_datetime(df['INIT DATE'], errors='coerce')

def extract_name(memberships):
    try:
        if memberships and isinstance(memberships, list):
            # Check if the list has at least one dictionary
            if len(memberships) > 0 and isinstance(memberships[0], dict):
                # Extract 'name' from the first dictionary in the list
                return memberships[0].get('section', {}).get('name', '')
        return ''  # Return empty string if conditions are not met
    except Exception as e:
        print(f"Error processing memberships: {e}")
        return ''  # Return empty string in case of any exception

# Apply the function to the 'memberships' column
df['STATUS'] = df['STATUS'].apply(lambda x: extract_name(x))


def extract_names(tag_list):
    if not tag_list:  # Check if the record is empty or None
        return []
    return [tag['name'] for tag in tag_list]  # Extract the 'name' values

df['tags'] = df['tags'].apply(lambda x: extract_names(x))


def determine_project_type(tags):
    if 'OLD' in tags:
        return 'OLD'
    elif 'NEW' in tags:
        return 'NEW'
    else:
        return '<no_project_type>'

df['PROJECT_TYPE'] = df['tags'].apply(lambda x: determine_project_type(x))


def determine_area_type(tags):
    if 'COFFEE' in tags:
        return 'COFFEE'
    elif 'MILK' in tags:
        return 'MILK'
    else:
        return '<no_area>'

df['AREA'] = df['tags'].apply(lambda x: determine_area_type(x))



def determine_type(tags):
    if 'TARGET' in tags:
        return 'TARGET'
    elif 'ADDITIONAL' in tags:
        return 'ADDITIONAL'
    else:
        return '<no_type>'

df['TYPE'] = df['tags'].apply(lambda x: determine_area_type(x))

def determine_doc_type(tags):
    if 'MPLAN' in tags:
        return 'MPLAN'
    elif 'AMM' in tags:
        return 'AMM'
    else:
        return '<no_doc_type>'

df['DOC_TYPE'] = df['tags'].apply(lambda x: determine_doc_type(x))


def determine_type(tags):
    if 'TARGET' in tags:
        return 'TARGET'
    elif 'ADDITIONAL' in tags:
        return 'ADDITIONAL'
    else:
        return '<no_type>'

df['TYPE'] = df['tags'].apply(lambda x: determine_area_type(x))




project_list = [
    "AUTOCOMPACTOR",
    "HYDRAULIC PROJECT",
    "GCU+",
    "WATER HEATER",
    "GCCD Transport Phase 1",
    "SIR",
    "IPTA VACUUM PUMP",
    "ICIP",
    "GC Van Unloading Facility",
    "PEC 2020: PE Panel Cooling",
    "PEC 2020: Process adaptation",
    "FFE-RARE",
    "PEC 2020: Spot Cooling"
]



# Function to find the project in tags
def find_project(tags):
    for project in project_list:
        if project in tags:
            return project
    return None  # or return an empty string "" if no match is found

# Apply the function to the tags column using lambda
df['PROJECT'] = df['tags'].apply(lambda x: find_project(x))



def calculate_age(init_date_str):
    # Convert the string date to a datetime object
    init_date = pd.to_datetime(init_date_str)
    # Get today's date
    today = datetime.now()
    # Calculate the difference in days
    age_days = (today - init_date).days
    return age_days

df['INIT_AGE (DAYS)'] = df['INIT DATE'].apply(lambda x: calculate_age(x))


def task_stories(row):
    return "Temporary REMARKS"

df['REMARKS'] = df.apply(lambda row: task_stories(row), axis=1)




#ADDING TIMESTAMP COLUMN
# Define the get_latest_created_at function
def get_latest_created_at(task_gid, stories_api_instance, opts):
    try:
        # Get stories from a task
        api_response = stories_api_instance.get_stories_for_task(task_gid, opts)
        
        assigned_time = None
        section_changed_time = None
        added_to_project_time = None
        
        for data in api_response:
            resource_subtype = data.get('resource_subtype')
            created_at = data.get('created_at')

            if created_at:
                created_at_date = datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%S.%fZ').date()

            if resource_subtype == 'assigned':
                assigned_time = created_at_date
            elif resource_subtype == 'section_changed':
                section_changed_time = created_at_date
            elif resource_subtype == 'added_to_project':
                added_to_project_time = created_at_date


        if assigned_time:
            return assigned_time.strftime('%Y-%m-%d')
        elif section_changed_time:
            return section_changed_time.strftime('%Y-%m-%d')
        elif added_to_project_time:
            return added_to_project_time.strftime('%Y-%m-%d')
        else:
            return None

    except asana.ApiException as e:
        print("Exception when calling StoriesApi->get_stories_for_task: %s\n" % e)
        return None

# Set up Asana API client and StoriesApi instance
stories_api_instance = asana.StoriesApi(api_client)
opts = {
    'opt_fields': "created_at,resource_subtype"
}

# Use lambda function to create the new 'TIMESTAMP' column in the DataFrame
df['TIMESTAMP'] = df['gid'].apply(lambda x: get_latest_created_at(x, stories_api_instance, opts))
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], errors='coerce')










def calculate_age_in_days(timestamp):
    if pd.isna(timestamp):
        return None
    # Directly use the date() method if timestamp is a Timestamp object
    timestamp_date = timestamp.date() if isinstance(timestamp, pd.Timestamp) else datetime.strptime(timestamp, '%Y-%m-%d').date()
    return (datetime.now().date() - timestamp_date).days

# Applying the function to create a new column 'AGE (DAYS)'
df['AGE (DAYS)'] = df['TIMESTAMP'].apply(calculate_age_in_days)




#DATA TYPES CONVERSION
columns_to_convert = [
    'PIC', 'STATUS', 'DESCRIPTION', 'PROJECT_TYPE',
    'AREA', 'TYPE', 'DOC_TYPE', 'PROJECT', 'REMARKS'
]

df[columns_to_convert] = df[columns_to_convert].astype(str)
df['gid'] = pd.to_numeric(df['gid'], errors='coerce').astype('Int64')  # Use 'Int64' to handle NaN values

df = df.drop(columns=['tags'])




#POPULATING REMARKS COLUMN
def get_remarks_for_task(task_gid):
    configuration = asana.Configuration()
    configuration.access_token = os.getenv('ASANA_TOKEN')
    api_client = asana.ApiClient(configuration)

    stories_api_instance = asana.StoriesApi(api_client)
    opts = {
        'opt_fields': "created_at,resource_subtype,text,created_by.name"
    }

    relevant_subtypes = {"added_to_project", "section_changed", "assigned", "unassigned", "comment_added"}
    remarks = []

    try:
        # Get stories from a task
        api_response = stories_api_instance.get_stories_for_task(task_gid, opts)
        
        for story in api_response:
            if story.get('resource_subtype') in relevant_subtypes:
                # Extract and format the required information
                created_at = story.get('created_at')[:10]  # Extract date only (YYYY-MM-DD)
                resource_subtype = story.get('resource_subtype')
                text = story.get('text')
                formatted_remark = f"[{created_at}] {resource_subtype}: \"{text}\""
                remarks.append(formatted_remark)
                
        
        # Check if no relevant stories were found
        if not remarks:
            remarks.append("<No Relevant Activity>")

    except ApiException as e:
        print("Exception when calling StoriesApi->get_stories_for_task: %s\n" % e)
        remarks.append("<No Relevant Activity>")

    return "\n".join(remarks)

# Example usage with lambda to populate the REMARKS column in the DataFrame
df['REMARKS'] = df['gid'].apply(lambda gid: get_remarks_for_task(gid))



#DISPLAYING DF
df

,gid,PIC,INIT DATE,STATUS,DESCRIPTION,PART / MPLAN NO.,PROJECT_TYPE,AREA,TYPE,DOC_TYPE,PROJECT,INIT_AGE (DAYS),REMARKS,TIMESTAMP,AGE (DAYS)
0,1208034272468521,Franciscarlo Tadena,2024-08-12,DONE,DOOR FLAP PNUEMATIC CYLINDER - REPAIR,,OLD,COFFEE,COFFEE,MPLAN,AUTOCOMPACTOR,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
1,1208034281348801,Franciscarlo Tadena,2024-08-12,DONE,14W ENERGY CHAIN - INSPECTION,,OLD,COFFEE,COFFEE,MPLAN,AUTOCOMPACTOR,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
2,1208034281348813,Franciscarlo Tadena,2024-08-12,DONE,6M DRIVE GEAR MOTOR - PM INSPECTION,,OLD,COFFEE,COFFEE,MPLAN,AUTOCOMPACTOR,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
3,1208034358879963,Franciscarlo Tadena,2024-08-12,DONE,3Y DRIVE GEAR MOTOR PM CHANGE GEAR OIL,,OLD,COFFEE,COFFEE,MPLAN,AUTOCOMPACTOR,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
4,1208034354473967,Franciscarlo Tadena,2024-08-12,DONE,3W DRIVE GEAR MOTOR - INSPECTION -,,OLD,COFFEE,COFFEE,MPLAN,AUTOCOMPACTOR,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,1208034570414673,R Suello,2024-08-12,WAITING FOR OTHER DOCS,<Dosusign to inform stakeholders of the unnece...,,OLD,COFFEE,COFFEE,AMM,PEC 2020: Spot Cooling,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
254,1208040839810255,Jennelyn Allado,2024-08-13,ON-GOING FIRST DRAFT,Test Task 2,,<no_project_type>,<no_area>,<no_area>,<no_doc_type>,None,14,"[2024-08-13] added_to_project: ""Francis Carlo ...",2024-08-27,0
255,1208034291082035,Franciscarlo Tadena,2024-08-12,ON-GOING FIRST DRAFT,LEMA 425 AZ AA1 4B : 20017164 - guide disc assy,,OLD,COFFEE,COFFEE,AMM,None,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15
256,1208034291082047,Franciscarlo Tadena,2024-08-12,ON-GOING FIRST DRAFT,LEMA 425 AZ AA1 4B : 20017131 - vane wheel imp...,,OLD,COFFEE,COFFEE,AMM,None,15,"[2024-08-12] added_to_project: ""Francis Carlo ...",2024-08-12,15


In [25]:


db_params = {
    'host': os.getenv('DB_HOST') or 'localhost',
    'database': os.getenv('DB_NAME') or 'tbmc_db',
    'user': os.getenv('DB_USER') or 'tbmc_db_user',
    'password': os.getenv('DB_PASSWORD') or '123456',
    'table': os.getenv('DB_TABLE') or 'tbmc_enrollment_general',
    'port': os.getenv('DB_PORT') or '5432'
}

def connect_to_database(db_params):
    try:
        conn = psycopg2.connect(
            host=db_params['host'],
            database=db_params['database'],
            user=db_params['user'],
            password=db_params['password']
        )
        conn.set_session(autocommit=True)
        
        engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")
        
        return conn, engine
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None, None
    
conn, engine = connect_to_database(db_params)

table_name = db_params['table']


def upload_to_database(df, engine, table_name):
    try:
        # Drop table if exists
        meta = MetaData()
        meta.reflect(bind=engine)
        
        if table_name in meta.tables:
            meta.tables[table_name].drop(engine, checkfirst=True)
            print(f"Table '{table_name}' dropped successfully (including dependent objects).")
        else:
            print(f"Table '{table_name}' does not exist. Proceeding to create a new table.")
        
        # Upload data to database
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Data uploaded successfully to table '{table_name}'.")
    except Exception as e:
        print(f"Error uploading data to database: {e}")
        

upload_to_database(df, engine, table_name)


conn.close()
print("Database connection closed.")

Error connecting to database: connection to server at "tbmcazureserverjapan.postgres.database.azure.com" (172.207.229.68), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?

Error uploading data to database: No inspection system is available for object of type <class 'NoneType'>


AttributeError: 'NoneType' object has no attribute 'close'

In [9]:

# gid_file_path = 'workspace.json'

# load_dotenv()
# access_token = os.getenv('ASANA_TOKEN')
# workspace_gid = get_gid_from_json(gid_file_path)


# configuration = asana.Configuration()
# configuration.access_token = access_token
# api_client = asana.ApiClient(configuration)


# # create an instance of the API class
# stories_api_instance = asana.StoriesApi(api_client)
# task_gid = "1208040839810255" # str | The task to operate on.
# opts = {
#     'opt_fields': "assignee,assignee.name,created_at,created_by,created_by.name,custom_field,custom_field.date_value,custom_field.date_value.date,custom_field.date_value.date_time,custom_field.display_value,custom_field.enabled,custom_field.enum_options,custom_field.enum_options.color,custom_field.enum_options.enabled,custom_field.enum_options.name,custom_field.enum_value,custom_field.enum_value.color,custom_field.enum_value.enabled,custom_field.enum_value.name,custom_field.id_prefix,custom_field.is_formula_field,custom_field.multi_enum_values,custom_field.multi_enum_values.color,custom_field.multi_enum_values.enabled,custom_field.multi_enum_values.name,custom_field.name,custom_field.number_value,custom_field.representation_type,custom_field.resource_subtype,custom_field.text_value,custom_field.type,dependency,dependency.created_by,dependency.name,dependency.resource_subtype,duplicate_of,duplicate_of.created_by,duplicate_of.name,duplicate_of.resource_subtype,duplicated_from,duplicated_from.created_by,duplicated_from.name,duplicated_from.resource_subtype,follower,follower.name,hearted,hearts,hearts.user,hearts.user.name,html_text,is_editable,is_edited,is_pinned,liked,likes,likes.user,likes.user.name,new_approval_status,new_date_value,new_dates,new_dates.due_at,new_dates.due_on,new_dates.start_on,new_enum_value,new_enum_value.color,new_enum_value.enabled,new_enum_value.name,new_multi_enum_values,new_multi_enum_values.color,new_multi_enum_values.enabled,new_multi_enum_values.name,new_name,new_number_value,new_people_value,new_people_value.name,new_resource_subtype,new_section,new_section.name,new_text_value,num_hearts,num_likes,offset,old_approval_status,old_date_value,old_dates,old_dates.due_at,old_dates.due_on,old_dates.start_on,old_enum_value,old_enum_value.color,old_enum_value.enabled,old_enum_value.name,old_multi_enum_values,old_multi_enum_values.color,old_multi_enum_values.enabled,old_multi_enum_values.name,old_name,old_number_value,old_people_value,old_people_value.name,old_resource_subtype,old_section,old_section.name,old_text_value,path,previews,previews.fallback,previews.footer,previews.header,previews.header_link,previews.html_text,previews.text,previews.title,previews.title_link,project,project.name,resource_subtype,source,sticker_name,story,story.created_at,story.created_by,story.created_by.name,story.resource_subtype,story.text,tag,tag.name,target,target.created_by,target.name,target.resource_subtype,task,task.created_by,task.name,task.resource_subtype,text,type,uri", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
# }

# try:
#     # Get stories from a task
#     api_response = stories_api_instance.get_stories_for_task(task_gid, opts)
#     for data in api_response:
#         pprint(data)
# except ApiException as e:
#     print("Exception when calling StoriesApi->get_stories_for_task: %s\n" % e)


{'created_at': '2024-08-13T01:42:46.527Z',
 'created_by': {'gid': '1207998336820177', 'name': 'Francis Carlo A. Tadena'},
 'gid': '1208041382776994',
 'html_text': '<b><a '
              'href="https://app.asana.com/0/profile/1207998439135094">Francis '
              'Carlo A. Tadena</a></b> added to <a '
              'href="https://app.asana.com/0/1207998440065110/1207998440065110">AMM '
              '&amp; MPLAN Enrollment</a>',
 'is_editable': False,
 'new_name': None,
 'previews': [],
 'project': {'gid': '1207998440065110', 'name': 'AMM & MPLAN Enrollment'},
 'resource_subtype': 'added_to_project',
 'source': 'web',
 'target': {'created_by': {'gid': '1207998336820177', 'resource_type': 'user'},
            'gid': '1208040839810255',
            'name': 'Test Task 2',
            'resource_subtype': 'default_task'},
 'text': 'Francis Carlo A. Tadena added to AMM & MPLAN Enrollment',
 'type': 'system'}
{'created_at': '2024-08-14T13:55:39.851Z',
 'created_by': {'gid': '1207998336820

In [ ]:
#SAMPLE HOW TO ADD 2 DATAFRAMES
# THE PURPOSE IS TO NOT USE THE LAMBDA FUNCTION FOR EACH RECORD TO POPULATE A COLUMN. 
# APPLY THESE TO THE TIMESTAMP COLUMN
# import pandas as pd

# # Merge the dataframes based on the matching task_gid column
# df_merged = pd.merge(df, remarks_table[['task_gid', 'story']], on='task_gid', how='left')

# # Rename the merged column as "remarks"
# df_merged.rename(columns={'story': 'remarks'}, inplace=True)

# # The df_merged dataframe now contains the added "remarks" column from remarks_table

In [ ]:
# import asana
# from asana.rest import ApiException
# from pprint import pprint

# configuration = asana.Configuration()
# configuration.access_token = '<YOUR_ACCESS_TOKEN>'
# api_client = asana.ApiClient(configuration)

# # create an instance of the API class
# tasks_api_instance = asana.TasksApi(api_client)
# opts = {
#     'limit': 50, # int | Results per page. The number of objects to return per page. The value must be between 1 and 100.
#     'offset': "eyJ0eXAiOJiKV1iQLCJhbGciOiJIUzI1NiJ9", # str | Offset token. An offset to the next page returned by the API. A pagination request will return an offset token, which can be used as an input parameter to the next request. If an offset is not passed in, the API will return the first page of results. *Note: You can only pass in an offset that was returned to you via a previously paginated request.*
#     'assignee': "14641", # str | The assignee to filter tasks on. If searching for unassigned tasks, assignee.any = null can be specified. *Note: If you specify `assignee`, you must also specify the `workspace` to filter on.*
#     'project': "321654", # str | The project to filter tasks on.
#     'section': "321654", # str | The section to filter tasks on.
#     'workspace': "321654", # str | The workspace to filter tasks on. *Note: If you specify `workspace`, you must also specify the `assignee` to filter on.*
#     'completed_since': '2012-02-22T02:06:58.158Z', # datetime | Only return tasks that are either incomplete or that have been completed since this time.
#     'modified_since': '2012-02-22T02:06:58.158Z', # datetime | Only return tasks that have been modified since the given time.  *Note: A task is considered “modified” if any of its properties change, or associations between it and other objects are modified (e.g.  a task being added to a project). A task is not considered modified just because another object it is associated with (e.g. a subtask) is modified. Actions that count as modifying the task include assigning, renaming, completing, and adding stories.*
#     'opt_fields': "actual_time_minutes,approval_status,assignee,assignee.name,assignee_section,assignee_section.name,assignee_status,completed,completed_at,completed_by,completed_by.name,created_at,created_by,custom_fields,custom_fields.asana_created_field,custom_fields.created_by,custom_fields.created_by.name,custom_fields.currency_code,custom_fields.custom_label,custom_fields.custom_label_position,custom_fields.date_value,custom_fields.date_value.date,custom_fields.date_value.date_time,custom_fields.description,custom_fields.display_value,custom_fields.enabled,custom_fields.enum_options,custom_fields.enum_options.color,custom_fields.enum_options.enabled,custom_fields.enum_options.name,custom_fields.enum_value,custom_fields.enum_value.color,custom_fields.enum_value.enabled,custom_fields.enum_value.name,custom_fields.format,custom_fields.has_notifications_enabled,custom_fields.id_prefix,custom_fields.is_formula_field,custom_fields.is_global_to_workspace,custom_fields.is_value_read_only,custom_fields.multi_enum_values,custom_fields.multi_enum_values.color,custom_fields.multi_enum_values.enabled,custom_fields.multi_enum_values.name,custom_fields.name,custom_fields.number_value,custom_fields.people_value,custom_fields.people_value.name,custom_fields.precision,custom_fields.representation_type,custom_fields.resource_subtype,custom_fields.text_value,custom_fields.type,dependencies,dependents,due_at,due_on,external,external.data,followers,followers.name,hearted,hearts,hearts.user,hearts.user.name,html_notes,is_rendered_as_separator,liked,likes,likes.user,likes.user.name,memberships,memberships.project,memberships.project.name,memberships.section,memberships.section.name,modified_at,name,notes,num_hearts,num_likes,num_subtasks,offset,parent,parent.created_by,parent.name,parent.resource_subtype,path,permalink_url,projects,projects.name,resource_subtype,start_at,start_on,tags,tags.name,uri,workspace,workspace.name", # list[str] | This endpoint returns a compact resource, which excludes some properties by default. To include those optional properties, set this query parameter to a comma-separated list of the properties you wish to include.
# }

# try:
#     # Get multiple tasks
#     api_response = tasks_api_instance.get_tasks(opts)
#     for data in api_response:
#         pprint(data)
# except ApiException as e:
#     print("Exception when calling TasksApi->get_tasks: %s\n" % e)

In [18]:
# df.dtypes[df.dtypes == 'object'].index  # This will list all columns with object dtype
# # Check the actual type of elements in these columns
# for column in df.dtypes[df.dtypes == 'object'].index:
#     print(f'{column}:', df[column].apply(type).unique())

PIC: [<class 'str'>]
STATUS: [<class 'str'>]
DESCRIPTION: [<class 'str'>]
PART / MPLAN NO.: [<class 'str'>]
PROJECT_TYPE: [<class 'str'>]
AREA: [<class 'str'>]
TYPE: [<class 'str'>]
DOC_TYPE: [<class 'str'>]
PROJECT: [<class 'str'>]
REMARKS: [<class 'str'>]
